# How to Use Furiosa SDK from Start to Finish

This notebook demonstrates how to use Furiosa SDK from start to finish.

## Prerequisites

The Furiosa SDK needs to have been installed. If not, it can be installed following instructions on https://furiosa-ai.github.io/docs/latest/ko/ (Korean) or https://furiosa-ai.github.io/docs/latest/en/ (English). The `torchvision` and `scipy` packages also need to be installed for this demonstration.

```console
$ pip install 'furiosa-sdk[quantizer]' torchvision scipy
```

In [1]:
import time

import numpy as np
import onnx
import torch
import torchvision
from torchvision import transforms
import tqdm

import furiosa.runtime.session
from furiosa.quantizer.frontend.onnx import post_training_quantize

libnpu.so --- v2.0, built @ e328545


## Load PyTorch Model

As a running example, we employ the pre-trained ReseNet-50 model from Torchvision.

In [2]:
torch_model = torchvision.models.resnet50(pretrained=True)
torch_model = torch_model.eval()  # Set the model to inference mode.

The ResNet50 model has been trained with the following preprocessing applied: https://pytorch.org/vision/stable/models.html We will use the same preprocessing for calibration and inference.

In [3]:
preprocess = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

## Export PyTorch Model to ONNX Model

We call the `torch.onnx.export` function to export the PyTorch ResNet-50 model to an ONNX model. The function executes a PyTorch model provided as its first argument, recording a trace of what operators are used during the execution, and then converts those operators into ONNX equivalents. Because `torch.onnx.export` runs the model, we need to provide the function with an input tensor as its second argument, which can be random so long as it satisfies the shape and type of the model's input. As of Furiosa SDK v0.6, ONNX OpSet 12 is the most well-supported version.

In [4]:
# Generate a dummy input of the shape, (1, 3, 224, 224), of the model's input.
dummy_input = (torch.randn(1, 3, 224, 224),)

# Export the PyTorch model into an ONNX model.
torch.onnx.export(
    torch_model,  # PyTorch model to export
    dummy_input,  # model input
    "resnet50.onnx",  # where to save the exported ONNX model
    opset_version=12,  # the ONNX OpSet version to export the model to
    do_constant_folding=True,  # whether to execute constant folding for optimization
    input_names=["input"],  # the ONNX model's input names
    output_names=["output"],  # the ONNX model's output names
)

# Load the exported ONNX model.
onnx_model = onnx.load_model("resnet50.onnx")

## Load Dataset

We will use subsets of the ImageNet dataset for calibration and validation. 

You need to download `ILSVRC2012_img_val.tar` and `ILSVRC2012_devkit_t12.tar.gz` externally and place them in the `imagenet` directory. Torchvision cannot download the ImageNet dataset automatically because it is no longer publicly accessible: https://github.com/pytorch/vision/pull/1457.

Note that it may take several minutes to run this step for the first time because it involves decompressing the archive files. It will take much less time to complete subsequently.

In [5]:
imagenet = torchvision.datasets.ImageNet("imagenet", split="val", transform=preprocess)

## Calibrate and Quantize ONNX Model

We call the `furiosa.quantizer.frontend.onnx.post_training_quantize` function to calibrate and quantize the ONNX model at one fell swoop. For quick demonstration, a small number of samples randomly chosen from the ImageNet dataset is used for calibration.

In [6]:
calibration_dataset = torch.utils.data.Subset(imagenet, torch.randperm(len(imagenet))[:100])
calibration_dataloader = torch.utils.data.DataLoader(calibration_dataset, batch_size=1)

onnx_model_quantized = post_training_quantize(
    onnx_model,
    ({"input": image.numpy()} for image, _ in calibration_dataloader),
)
onnx.save_model(onnx_model_quantized, "resnet50_quantized.onnx")

Calibration: 100it [00:22,  4.48it/s]
Quantization: 100%|███████████████████████████████████████████████| 122/122 [00:02<00:00, 50.59it/s]


## Run Inference with Quantized ONNX Model

For quick demonstration, we use randomly chosen 1000 samples from the ImageNet dataset for validation.

In [7]:
validation_dataset = torch.utils.data.Subset(imagenet, torch.randperm(len(imagenet))[:1000])
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=1)

correct_predictions, total_predictions = 0, 0
elapsed_time = 0
with furiosa.runtime.session.create("resnet50_quantized.onnx") as session:
    for image, label in tqdm.tqdm(validation_dataloader, desc="Evaluation", unit="images", mininterval=0.5):
        image = image.numpy()
        start = time.perf_counter_ns()
        outputs = session.run(image)
        elapsed_time += time.perf_counter_ns() - start
        
        prediction = np.argmax(outputs[0].numpy(), axis=1)  # postprocessing  
        if prediction == label.numpy():
            correct_predictions += 1
        total_predictions += 1

Saving the compilation log into /root/.local/state/furiosa/logs/compile-20220325011645-slh1k0.log
Using furiosa-compiler 0.6.1 (rev: 6edff7711 built at 2022-03-22 18:51:51)
2022-03-25T01:16:45.038999Z  INFO Npu (npu6pe0-1) is being initialized
2022-03-25T01:16:45.039809Z  INFO NuxInner create with pes: [PeId(0)]


[1/6] 🔍   Compiling from onnx to dfg
Done in 0.040535457s
[2/6] 🔍   Compiling from dfg to ldfg
Done in 341.40164s
[3/6] 🔍   Compiling from ldfg to cdfg
Done in 0.001958493s
[4/6] 🔍   Compiling from cdfg to gir
Done in 0.022858769s
[5/6] 🔍   Compiling from gir to lir
Done in 0.005438614s
[6/6] 🔍   Compiling from lir to enf
Done in 0.064078905s
✨  Finished in 341.53796s


2022-03-25T01:22:26.606689Z  INFO [Profiler] Program binary notification has been arrived. Cleanup current profile queue data


Evaluation: 100%|███████████████████████████████████████████| 1000/1000 [00:22<00:00, 44.88images/s]

2022-03-25T01:22:49.489470Z  INFO [Profiler] Received a termination signal.


2022-03-25T01:22:49.490451Z  INFO session has been destroyed


In [8]:
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy:%}")

latency = elapsed_time / total_predictions
print(f"Average Latency: {latency / 1_000_000} ms")

Accuracy: 76.200000%
Average Latency: 8.689881424000001 ms
